# simpy model results collection - advanced methods

In this notebook you will learn

* How to create a monitored `simpy.Resource`
* How to use the observer pattern to calculate running totals of metrics during a simulation run.

In [3]:
import simpy
import numpy as np

simpy.__version__

'4.0.1'

## Monitoring a resource

A `simpy.Resource` is a python class.  You can inherit from this class and override the `request()` and `release()` methods.  This enables you to record the historical number of processes queuing for a resource and in progress OR calculate a online statistic such as the mean or variance in an event driven approach.

The class `MonitoredResource` inherits from `simpy.Resource`.  The methods `__init__`, `request()` and `release()` are overriden in a simple way.  

The mean queue length is updated each time a resource is requested using the formula

$L_q^t = \dfrac{n-1}{n}L_{q}^{t-1} + \dfrac{1}{n}q_t$

In [5]:
class MonitoredResource(simpy.Resource):
    '''
    adapted from: 
    https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html
    '''
    def __init__(self, *args, **kwargs):
        #super() is the super class i.e. simpy.Resource 
        super().__init__(*args, **kwargs)
        
        #the number of arrivals to the resource
        self.n = 0
        
        #the mean number queuing for service
        self.mean_queue_length = 0.0
        self.mean_queue = 0.0
        
        #mean service
        self.mean_in_service = 0.0
        
    def request(self, *args, **kwargs):
        
        #new arrival
        self.n += 1
        
        #update the mean queue length
        self.mean_queue_length = ((self.n - 1) / float(self.n)) * \
            self.mean_queue_length + (1 / float(self.n)) * len(self.queue)
        
        self.mean_queue += (len(self.queue) - self.mean_queue) / self.n
        
        #update the mean in service
        self.mean_in_service = ((self.n - 1) / float(self.n)) * \
            self.mean_in_service + (1 / float(self.n)) * self.count
        
        return super().request(*args, **kwargs)